In [1]:
%cd /content
!wget -c https://alphacephei.com/vosk-colab/kaldi.tar.gz
!tar xzf kaldi.tar.gz

/content
--2022-08-17 10:53:18--  https://alphacephei.com/vosk-colab/kaldi.tar.gz
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 809174554 (772M) [application/octet-stream]
Saving to: ‘kaldi.tar.gz’

kaldi.tar.gz        100%[===================>] 771.69M   108MB/s    in 7.2s    

2022-08-17 10:53:25 (106 MB/s) - ‘kaldi.tar.gz’ saved [809174554/809174554]



In [9]:
%cd /content
!rm -rf vosk-api
!git clone https://github.com/alphacep/vosk-api

/content
Cloning into 'vosk-api'...
remote: Enumerating objects: 3259, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 3259 (delta 45), reused 92 (delta 28), pack-reused 3135
Receiving objects: 100% (3259/3259), 1.07 MiB | 8.46 MiB/s, done.
Resolving deltas: 100% (1847/1847), done.


In [10]:
!rm -rf /content/kaldi/egs/ac/training
!cp -r /content/vosk-api/training /content/kaldi/egs/ac

In [6]:
!apt install flac

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  flac
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 144 kB of archives.
After this operation, 438 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 flac amd64 1.3.2-1 [144 kB]
Fetched 144 kB in 0s (294 kB/s)
Selecting previously unselected package flac.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../flac_1.3.2-1_amd64.deb ...
Unpacking flac (1.3.2-1) ...
Setting up flac (1.3.2-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [12]:
%cd /content/kaldi/egs/ac/training
!rm -rf exp
!sed -i 's:--nj 10:--nj 2:g' run.sh
!cat run.sh
!bash run.sh --stop-stage 3

/content/kaldi/egs/ac/training
#!/usr/bin/env bash

. ./cmd.sh
. ./path.sh

stage=-1
stop_stage=100

. utils/parse_options.sh

# Data preparation
if [ ${stage} -le 0 ] && [ ${stop_stage} -ge 0 ]; then
  data_url=www.openslr.org/resources/31
  lm_url=www.openslr.org/resources/11
  database=corpus

  mkdir -p $database
  for part in dev-clean-2 train-clean-5; do
    local/download_and_untar.sh $database $data_url $part
  done

  local/download_lm.sh $lm_url $database data/local/lm

  local/data_prep.sh $database/LibriSpeech/train-clean-5 data/train
  local/data_prep.sh $database/LibriSpeech/dev-clean-2 data/test
fi

# Dictionary formatting
if [ ${stage} -le 1 ] && [ ${stop_stage} -ge 1 ]; then
  local/prepare_dict.sh data/local/lm data/local/dict
  utils/prepare_lang.sh data/local/dict "<UNK>" data/local/lang data/lang
fi

# Extract MFCC features
if [ ${stage} -le 2 ] && [ ${stop_stage} -ge 2 ]; then
  for task in train; do
    steps/make_mfcc.sh --cmd "$train_cmd" --nj 2 data/$task exp/

In [18]:
!sed -i 's:--nj 10:--nj 2:g' local/chain/run_tdnn.sh
!sed -i 's:steps/nnet3/chain/train.py:steps/nnet3/chain/train.py --use-gpu false:g' local/chain/run_tdnn.sh
!cat local/chain/run_tdnn.sh
!bash local/chain/run_tdnn.sh

#!/bin/bash

# Set -e here so that we catch if any executable fails immediately
set -euo pipefail

# (some of which are also used in this script directly).
stage=-1
decode_nj=10
train_set=train
gmm=tri3
nnet3_affix=
suffix=

# The rest are configs specific to this script.  Most of the parameters
# are just hardcoded at this level, in the commands below.
affix=   # affix for the TDNN directory name
tree_affix=
train_stage=-10
get_egs_stage=-10
decode_iter=

# training options
# training chunk-options
chunk_width=140,100,160
common_egs_dir=
xent_regularize=0.1
dropout_schedule='0,0@0.20,0.5@0.50,0'

# training options
srand=0
remove_egs=true

# End configuration section.
echo "$0 $@"  # Print the command line for logging

. ./cmd.sh
. ./path.sh
. ./utils/parse_options.sh

# Problem: We have removed the "train_" prefix of our training set in
# the alignment directory names! Bad!
gmm_dir=exp/$gmm
ali_dir=exp/${gmm}_ali
tree_dir=exp/chain${suffix}/tree${tree_affix:+_$tree_affix}
lang=data/l

In [19]:
!cp exp/chain/tdnn/13.mdl exp/chain/tdnn/final.mdl

In [23]:
!bash run.sh --stage 5

Converting 'data/local/lm/lm_tgsmall.arpa.gz' to FST
arpa2fst --disambig-symbol=#0 --read-symbol-table=data/lang_test/words.txt - data/lang_test/G.fst 
LOG (arpa2fst[5.5.1046~1-76cd5]:Read():arpa-file-parser.cc:94) Reading \data\ section.
LOG (arpa2fst[5.5.1046~1-76cd5]:Read():arpa-file-parser.cc:149) Reading \1-grams: section.
LOG (arpa2fst[5.5.1046~1-76cd5]:Read():arpa-file-parser.cc:149) Reading \2-grams: section.
LOG (arpa2fst[5.5.1046~1-76cd5]:Read():arpa-file-parser.cc:149) Reading \3-grams: section.
LOG (arpa2fst[5.5.1046~1-76cd5]:RemoveRedundantStates():arpa-lm-compiler.cc:359) Reduced num-states from 1183418 to 249533
fstisstochastic data/lang_test/G.fst 
9.90926e-06 -0.29805
Succeeded in formatting LM: 'data/local/lm/lm_tgsmall.arpa.gz'
tree-info exp/chain/tdnn/tree 
tree-info exp/chain/tdnn/tree 
fstdeterminizestar --use-log=true 
fsttablecompose data/lang_test/L_disambig.fst data/lang_test/G.fst 
fstpushspecial 
fstminimizeencoded 
fstisstochastic data/lang_test/tmp/LG.fst 